# Evaluierung der LLM-Pipeline für Pressemitteilungsgenerierung

Dieses Notebook demonstriert die Verwendung der Evaluierungspipeline für verschiedene LLMs zur Generierung von Pressemitteilungen aus Gerichtsurteilen mit Hilfe synthetischer Prompts.

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Projekt-spezifische Importe
from courtpressger.evaluation.pipeline import LLMEvaluationPipeline
from courtpressger.evaluation.models import create_model_config
from courtpressger.evaluation.utils import (
    load_evaluation_results,
    results_to_dataframe,
    visualize_rouge_scores,
    visualize_metric_comparison,
    extract_top_examples,
    compute_statistical_significance
)

# Stile für schönere Visualisierungen setzen
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Konfiguration und Datenvorbereitung

Zuerst laden wir die Modellkonfiguration und den Evaluierungsdatensatz.

In [ ]:
# Pfade definieren
PROJECT_ROOT = Path(os.getcwd()).parent
CONFIG_PATH = PROJECT_ROOT / "models" / "evaluation_models_config.json"
DATA_PATH = PROJECT_ROOT / "data" / "processed" / "evaluation_dataset.csv"  # Pfad anpassen
OUTPUT_DIR = PROJECT_ROOT / "data" / "evaluation"

# Ausgabeverzeichnis erstellen, falls nicht vorhanden
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Modellkonfiguration laden
with open(CONFIG_PATH, 'r') as f:
    config_data = json.load(f)
    models_config = config_data.get('models', [])

print(f"Geladen: {len(models_config)} Modelle aus der Konfiguration")

In [ ]:
# Testdaten oder tatsächliche Daten laden
# Falls die CSV-Datei nicht existiert, verwenden wir einen kleinen Beispieldatensatz
try:
    dataset = pd.read_csv(DATA_PATH)
    print(f"Datensatz geladen: {len(dataset)} Einträge")
except FileNotFoundError:
    print(f"Datei {DATA_PATH} nicht gefunden. Verwende einen Testdatensatz.")
    
    # Wir können alternativ einen Checkpoint-Datensatz laden
    checkpoint_files = list((PROJECT_ROOT / "data" / "checkpoints").glob("cases_prs_synth_prompts_*.csv"))
    if checkpoint_files:
        # Den ersten Checkpoint-Datensatz verwenden
        dataset = pd.read_csv(checkpoint_files[0])
        print(f"Checkpoint-Datensatz geladen: {len(dataset)} Einträge aus {checkpoint_files[0]}")
    else:
        # Oder einen einfachen Testdatensatz erstellen
        dataset = pd.DataFrame({
            'synth_prompt': ["Erstelle eine Pressemitteilung für das folgende Urteil:"],
            'ruling_text': ["Das Gericht hat entschieden..."],
            'press_release': ["In einer heutigen Entscheidung hat das Gericht..."]
        })
        print(f"Testdatensatz erstellt mit {len(dataset)} Einträgen")

# Spalten für die Evaluierung definieren
prompt_column = 'synth_prompt'  # Anpassen an tatsächliche Spaltenbezeichnung
ruling_column = 'ruling_text'   # Anpassen an tatsächliche Spaltenbezeichnung
press_column = 'press_release'  # Anpassen an tatsächliche Spaltenbezeichnung

# Einen ersten Blick auf die Daten werfen
dataset.head(2)

## 2. Modellkonfiguration

Wir bereiten die Modellkonfiguration für die Evaluierungspipeline vor. Da die API-Aufrufe zeitaufwändig und möglicherweise kostspielig sind, bieten wir die Option, entweder vorhandene Ergebnisse zu laden oder die Pipeline tatsächlich auszuführen.

In [ ]:
# Modellkonfiguration vorbereiten und Generatorfunktionen erstellen
prepared_models = []

for model_config in models_config:
    try:
        model_info = create_model_config(model_config)
        prepared_models.append(model_info)
        print(f"Modell konfiguriert: {model_info['name']}")
    except Exception as e:
        print(f"Fehler bei der Konfiguration von {model_config.get('name', 'Unbekannt')}: {str(e)}")

# Für dieses Notebook können wir optional eine Teilmenge der Modelle auswählen
# Beispiel: Nur die ersten beiden Modelle verwenden
# selected_models = prepared_models[:2]  # Auskommentieren, um alle Modelle zu verwenden
selected_models = prepared_models  # Alle konfigurierten Modelle verwenden

## 3. Evaluierungspipeline ausführen oder vorhandene Ergebnisse laden

Hier können wir entweder die Pipeline tatsächlich ausführen (was zeitaufwändig sein kann) oder vorhandene Ergebnisse laden, wenn die Evaluation bereits durchgeführt wurde.

In [ ]:
# Prüfen, ob Ergebnisse bereits vorhanden sind
evaluation_summary_path = OUTPUT_DIR / "evaluation_summary.json"
load_existing_results = evaluation_summary_path.exists()

if load_existing_results:
    print(f"Lade vorhandene Evaluierungsergebnisse aus {evaluation_summary_path}")
    results = load_evaluation_results(str(OUTPUT_DIR))
else:
    print("Führe Evaluierungspipeline aus (dies kann einige Zeit dauern)...")
    
    # Nur eine kleine Teilmenge des Datensatzes verwenden, falls der Datensatz groß ist
    if len(dataset) > 10:
        eval_dataset = dataset.sample(10, random_state=42)  # 10 zufällige Einträge für Demo
        print(f"Verwende 10 zufällige Einträge für die Demo-Evaluation")
    else:
        eval_dataset = dataset
    
    # Pipeline initialisieren und ausführen
    pipeline = LLMEvaluationPipeline(selected_models, output_dir=str(OUTPUT_DIR))
    pipeline_results = pipeline.run_evaluation(
        dataset=eval_dataset,
        prompt_column=prompt_column,
        ruling_column=ruling_column,
        reference_press_column=press_column,
        batch_size=5,  # Kleine Batch-Größe für Demo
        checkpoint_freq=5  # Häufige Checkpoints für Demo
    )
    
    # Ergebnisse laden
    results = load_evaluation_results(str(OUTPUT_DIR))

## 4. Ergebnisanalyse und Visualisierung

Jetzt analysieren und visualisieren wir die Evaluierungsergebnisse.

In [ ]:
# Ergebnisse in DataFrame konvertieren für einfachere Analyse
results_df = results_to_dataframe(results)

# Überblick über die Ergebnisse
print("Zusammenfassung der Evaluierungsergebnisse:")
results_df

In [ ]:
# ROUGE-Scores visualisieren
plt.figure(figsize=(14, 8))
visualize_rouge_scores(results_df)
plt.title("ROUGE-Scores nach Modell", fontsize=16)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "rouge_scores_comparison.png", dpi=300)
plt.show()

In [ ]:
# Vergleich aller verfügbaren Metriken (Radar-Chart)
metrics = [col for col in results_df.columns if col != "model"]
plt.figure(figsize=(12, 12))
visualize_metric_comparison(results_df, metrics)
plt.title("Vergleich aller Metriken nach Modell", fontsize=16)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "metrics_radar_comparison.png", dpi=300)
plt.show()

In [ ]:
# Zusätzliche Visualisierung: Heatmap der Metriken
plt.figure(figsize=(14, 10))
heatmap_data = results_df.set_index('model')
sns.heatmap(heatmap_data, annot=True, cmap="YlGnBu", fmt=".3f", linewidths=.5)
plt.title("Heatmap der Evaluierungsmetriken nach Modell", fontsize=16)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "metrics_heatmap.png", dpi=300)
plt.show()

## 5. Detaillierte Analyse einzelner Modelle

Wir können auch in die Ergebnisse einzelner Modelle eintauchen, um deren Stärken und Schwächen besser zu verstehen.

In [ ]:
# Eine Funktion, um die besten und schlechtesten Beispiele für ein Modell zu extrahieren
def analyze_model_examples(model_name):
    print(f"\n=== Analyse für Modell: {model_name} ===\n")
    
    # Beste Beispiele nach ROUGE-L
    print("Top 3 beste Beispiele (nach ROUGE-L F1-Score):")
    best_examples = extract_top_examples(results, model_name, "rougeL_fmeasure", top_n=3, highest=True)
    
    for i, example in enumerate(best_examples, 1):
        print(f"\nBeispiel {i} (Score: {example['score']:.4f})")
        print("\nReferenztext (Ausschnitt):")
        print(example['reference_text'][:200] + "..." if len(example['reference_text']) > 200 else example['reference_text'])
        print("\nGenerierter Text (Ausschnitt):")
        print(example['generated_text'][:200] + "..." if len(example['generated_text']) > 200 else example['generated_text'])
    
    # Schlechteste Beispiele nach ROUGE-L
    print("\n\nTop 3 schlechteste Beispiele (nach ROUGE-L F1-Score):")
    worst_examples = extract_top_examples(results, model_name, "rougeL_fmeasure", top_n=3, highest=False)
    
    for i, example in enumerate(worst_examples, 1):
        print(f"\nBeispiel {i} (Score: {example['score']:.4f})")
        print("\nReferenztext (Ausschnitt):")
        print(example['reference_text'][:200] + "..." if len(example['reference_text']) > 200 else example['reference_text'])
        print("\nGenerierter Text (Ausschnitt):")
        print(example['generated_text'][:200] + "..." if len(example['generated_text']) > 200 else example['generated_text'])

In [ ]:
# Beispielanalyse für ein bestimmtes Modell durchführen
if results_df is not None and not results_df.empty:
    model_to_analyze = results_df['model'].iloc[0]  # Erstes Modell in den Ergebnissen
    analyze_model_examples(model_to_analyze)

## 6. Statistische Signifikanz

Wir können auch testen, ob die Unterschiede zwischen den Modellen statistisch signifikant sind.

In [ ]:
# Statistische Signifikanz der Unterschiede zwischen Modellen berechnen
try:
    significance_df = compute_statistical_significance(results, metric="rougeL_fmeasure")
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(significance_df, annot=True, cmap="YlGnBu", fmt=".3f")
    plt.title("P-Werte für paarweise Vergleiche (ROUGE-L F1)", fontsize=16)
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "significance_heatmap.png", dpi=300)
    plt.show()
    
    print("\nStatistische Signifikanz (p-Werte):")
    print("Werte < 0.05 deuten auf signifikante Unterschiede hin\n")
    print(significance_df)
except Exception as e:
    print(f"Fehler bei der Berechnung der statistischen Signifikanz: {str(e)}")

## 7. Zusammenfassung

Hier fassen wir die wichtigsten Erkenntnisse aus der Evaluierung zusammen.

In [ ]:
# Rangliste der Modelle nach ROUGE-L F1-Score
if 'rougeL' in results_df.columns:
    ranked_models = results_df.sort_values(by='rougeL', ascending=False)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x='model', y='rougeL', data=ranked_models)
    plt.title("Rangliste der Modelle nach ROUGE-L F1-Score", fontsize=16)
    plt.xlabel("Modell")
    plt.ylabel("ROUGE-L F1-Score")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "model_ranking.png", dpi=300)
    plt.show()
    
    print("\nRangliste der Modelle nach ROUGE-L F1-Score:")
    for i, (idx, row) in enumerate(ranked_models.iterrows(), 1):
        print(f"{i}. {row['model']}: {row.get('rougeL', 'N/A'):.4f}")

## Fazit und nächste Schritte

In diesem Notebook haben wir die Evaluierungspipeline für verschiedene LLMs zur Generierung von Pressemitteilungen aus Gerichtsurteilen ausgeführt und visualisiert. Die Ergebnisse zeigen die Leistung der verschiedenen Modelle anhand mehrerer Metriken, insbesondere ROUGE-Scores.

**Erkenntnisse:**
- [Hier können Erkenntnisse zur Modellleistung ergänzt werden]
- [Vergleich der Modelle und ihre jeweiligen Stärken/Schwächen]

**Nächste Schritte:**
- Erweitern der Evaluierung um zusätzliche Metriken (z.B. BERTScore, menschliche Bewertungen)
- Optimierung der Prompts für die besten Modelle
- Detailliertere Fehleranalyse der schlechtesten Fälle
- Anpassung der Generierungsparameter (Temperatur, max_tokens, etc.)